# First look at data and data preparation

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
%matplotlib inline

In [2]:
path_train = "titanic/train.csv"
path_test = "titanic/test.csv"

In [3]:
train_data = pd.read_csv(path_train)

In [4]:
train_data.head(3)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S


#### We have two Id columns, one to many, we are going to set PassengerId column as index column

In [5]:
train_data.set_index('PassengerId',inplace = True)

In [6]:
train_data.tail(3)

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,,,
889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.45,NaN,S
890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.00,C148,C
891,0,3,"Dooley, Mr. Patrick",male,32.0,0,0,370376,7.75,NaN,Q


#### We must do the same with the test data:

In [7]:
test_data = pd.read_csv(path_test)

In [8]:
test_data.head(3)

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q


In [9]:
test_data.set_index('PassengerId',inplace = True)

In [10]:
test_data.head(3)

,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,,
892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q


# Exploring features

#### In reality we don't use test dataset at all, because test data is new, never seen data.

In [12]:
all_data = pd.concat([train_data,test_data])

In [13]:
print(train_data.shape,test_data.shape)
print(all_data.shape)

(891, 11) (418, 10)
(1309, 11)


In [14]:
all_data.tail(3)

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,,,
1307,NaN,3,"Saether, Mr. Simon Sivertsen",male,38.5,0,0,SOTON/O.Q. 3101262,7.2500,NaN,S
1308,NaN,3,"Ware, Mr. Frederick",male,NaN,0,0,359309,8.0500,NaN,S
1309,NaN,3,"Peter, Master. Michael J",male,NaN,1,1,2668,22.3583,NaN,C


#### We see that number of columns in train_data is 11 and in test_data is 10, it is due to fact that test_data does not have Survived column, we must predict that values.

In [15]:
all_data.isnull().sum()

Survived     418
Pclass         0
Name           0
Sex            0
Age          263
SibSp          0
Parch          0
Ticket         0
Fare           1
Cabin       1014
Embarked       2
dtype: int64

In [16]:
all_data['Age'].isnull().sum()

263

#### We have two columns, Age and Cabin with missing values. We are not going to easily input missing age values with mean or median value, later we are going to see why.

## Ticktet column: Getting ticket number from tickets

In [17]:
ticket = all_data['Ticket'].str.split()

In [18]:
ticket_numbers = ticket.str.get(-1)

In [19]:
ticket_numbers

PassengerId
1         21171
2         17599
3       3101282
4        113803
5        373450
         ...   
1305       3236
1306      17758
1307    3101262
1308     359309
1309       2668
Name: Ticket, Length: 1309, dtype: object

In [20]:
all_data['Ticket Number'] = ticket_numbers

In [21]:
all_data.head()

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Ticket Number
PassengerId,,,,,,,,,,,,
1,0.0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S,21171
2,1.0,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,17599
3,1.0,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,3101282
4,1.0,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,113803
5,0.0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S,373450


In [22]:
all_data['Ticket Number'] = pd.to_numeric(all_data['Ticket Number'])

ValueError: Unable to parse string "LINE" at position 179

In [23]:
all_data[178:180]

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Ticket Number
PassengerId,,,,,,,,,,,,
179,0.0,2,"Hale, Mr. Reginald",male,30.0,0,0,250653,13.0,NaN,S,250653
180,0.0,3,"Leonard, Mr. Lionel",male,36.0,0,0,LINE,0.0,NaN,S,LINE


There is passenger with Ticket "LINE", we will set his ticket number to 0

In [24]:
all_data.loc[180,'Ticket Number']=0

In [25]:
all_data['Ticket Number'] = pd.to_numeric(all_data['Ticket Number'])

ValueError: Unable to parse string "LINE" at position 271

#### There are more passengers with ticket number "LINE". Lets find them all:

In [26]:
all_data[all_data['Ticket Number']=='LINE']

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Ticket Number
PassengerId,,,,,,,,,,,,
272,1.0,3,"Tornquist, Mr. William Henry",male,25.0,0,0,LINE,0.0,NaN,S,LINE
303,0.0,3,"Johnson, Mr. William Cahoone Jr",male,19.0,0,0,LINE,0.0,NaN,S,LINE
598,0.0,3,"Johnson, Mr. Alfred",male,49.0,0,0,LINE,0.0,NaN,S,LINE


In [27]:
all_data.loc[272,'Ticket Number']=1
all_data.loc[303,'Ticket Number']=2
all_data.loc[598,'Ticket Number']=3

In [28]:
all_data['Ticket Number'] = pd.to_numeric(all_data['Ticket Number'])

#### Let's drop column Ticket:

In [29]:
all_data.drop('Ticket',axis = 1, inplace=True )

## Name column

In [30]:
all_data.query('SibSp>7')

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked,Ticket Number
PassengerId,,,,,,,,,,,
160,0.0,3,"Sage, Master. Thomas Henry",male,NaN,8,2,69.55,NaN,S,2343
181,0.0,3,"Sage, Miss. Constance Gladys",female,NaN,8,2,69.55,NaN,S,2343
202,0.0,3,"Sage, Mr. Frederick",male,NaN,8,2,69.55,NaN,S,2343
325,0.0,3,"Sage, Mr. George John Jr",male,NaN,8,2,69.55,NaN,S,2343
793,0.0,3,"Sage, Miss. Stella Anna",female,NaN,8,2,69.55,NaN,S,2343
847,0.0,3,"Sage, Mr. Douglas Bullen",male,NaN,8,2,69.55,NaN,S,2343
864,0.0,3,"Sage, Miss. Dorothy Edith ""Dolly""",female,NaN,8,2,69.55,NaN,S,2343
1080,NaN,3,"Sage, Miss. Ada",female,NaN,8,2,69.55,NaN,S,2343
1252,NaN,3,"Sage, Master. William Henry",male,14.5,8,2,69.55,NaN,S,2343


#### Large familes died in acident. Let's make new feture Family Name:

In [36]:
family = all_data['Name'].str.split(',')

In [35]:
all_data.head()

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked,Ticket Number
PassengerId,,,,,,,,,,,
1,0.0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,7.2500,NaN,S,21171
2,1.0,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,71.2833,C85,C,17599
3,1.0,3,"Heikkinen, Miss. Laina",female,26.0,0,0,7.9250,NaN,S,3101282
4,1.0,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,53.1000,C123,S,113803
5,0.0,3,"Allen, Mr. William Henry",male,35.0,0,0,8.0500,NaN,S,373450


In [37]:
all_data['Family Name'] = family.str.get(0)

In [38]:
all_data.head()

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked,Ticket Number,Family Name
PassengerId,,,,,,,,,,,,
1,0.0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,7.2500,NaN,S,21171,Braund
2,1.0,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,71.2833,C85,C,17599,Cumings
3,1.0,3,"Heikkinen, Miss. Laina",female,26.0,0,0,7.9250,NaN,S,3101282,Heikkinen
4,1.0,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,53.1000,C123,S,113803,Futrelle
5,0.0,3,"Allen, Mr. William Henry",male,35.0,0,0,8.0500,NaN,S,373450,Allen


In [39]:
all_data[all_data['Family Name']=='Sage']

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked,Ticket Number,Family Name
PassengerId,,,,,,,,,,,,
160,0.0,3,"Sage, Master. Thomas Henry",male,NaN,8,2,69.55,NaN,S,2343,Sage
181,0.0,3,"Sage, Miss. Constance Gladys",female,NaN,8,2,69.55,NaN,S,2343,Sage
202,0.0,3,"Sage, Mr. Frederick",male,NaN,8,2,69.55,NaN,S,2343,Sage
325,0.0,3,"Sage, Mr. George John Jr",male,NaN,8,2,69.55,NaN,S,2343,Sage
793,0.0,3,"Sage, Miss. Stella Anna",female,NaN,8,2,69.55,NaN,S,2343,Sage
847,0.0,3,"Sage, Mr. Douglas Bullen",male,NaN,8,2,69.55,NaN,S,2343,Sage
864,0.0,3,"Sage, Miss. Dorothy Edith ""Dolly""",female,NaN,8,2,69.55,NaN,S,2343,Sage
1080,NaN,3,"Sage, Miss. Ada",female,NaN,8,2,69.55,NaN,S,2343,Sage
1234,NaN,3,"Sage, Mr. John George",male,NaN,1,9,69.55,NaN,S,2343,Sage


#### Also, we can see that beside Family Name, inside Name column we have title: Mr, Mrs, Ms, Master, etc. Let's export that in new column

In [40]:
title = family.str.get(1).str.split('.').str.get(0)

In [41]:
all_data['Title'] = title.str.strip()

In [42]:
all_data.head()

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked,Ticket Number,Family Name,Title
PassengerId,,,,,,,,,,,,,
1,0.0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,7.2500,NaN,S,21171,Braund,Mr
2,1.0,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,71.2833,C85,C,17599,Cumings,Mrs
3,1.0,3,"Heikkinen, Miss. Laina",female,26.0,0,0,7.9250,NaN,S,3101282,Heikkinen,Miss
4,1.0,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,53.1000,C123,S,113803,Futrelle,Mrs
5,0.0,3,"Allen, Mr. William Henry",male,35.0,0,0,8.0500,NaN,S,373450,Allen,Mr


#### Let's see age distribution over the Titles values

In [43]:
all_data.pivot_table('Age',index='Title',aggfunc=['min','mean','max','count'])

,min,mean,max,count
,Age,Age,Age,Age
Title,,,,
Capt,70.00,70.000000,70.0,1
Col,47.00,54.000000,60.0,4
Don,40.00,40.000000,40.0,1
Dona,39.00,39.000000,39.0,1
Dr,23.00,43.571429,54.0,7
Jonkheer,38.00,38.000000,38.0,1
Lady,48.00,48.000000,48.0,1
Major,45.00,48.500000,52.0,2


### There is too much title values that can be grouped together

In [44]:
all_data['Title'] = all_data['Title'].replace(['Mlle','Mme','Ms'],'Miss')

In [45]:
all_data['Title'] = all_data['Title'].replace(['Major','Capt','Col', 'Don','Jonkheer'],'Mr')

In [46]:
all_data.pivot_table('Age',index='Title',aggfunc=['min','mean','max','count'])

,min,mean,max,count
,Age,Age,Age,Age
Title,,,,
Dona,39.00,39.000000,39.0,1
Dr,23.00,43.571429,54.0,7
Lady,48.00,48.000000,48.0,1
Master,0.33,5.482642,14.5,53
Miss,0.17,21.834533,63.0,214
Mr,11.00,32.541525,80.0,590
Mrs,14.00,36.994118,76.0,170
Rev,27.00,41.250000,57.0,8


In [47]:
all_data['Title'] = all_data['Title'].replace(['Dona'],'Mrs')

In [48]:
all_data[all_data.Title == 'Lady']

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked,Ticket Number,Family Name,Title
PassengerId,,,,,,,,,,,,,
557,1.0,1,"Duff Gordon, Lady. (Lucille Christiana Sutherl...",female,48.0,1,0,39.6,A16,C,11755,Duff Gordon,Lady


In [49]:
all_data[all_data['Family Name']== 'Duff Gordon']

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked,Ticket Number,Family Name,Title
PassengerId,,,,,,,,,,,,,
557,1.0,1,"Duff Gordon, Lady. (Lucille Christiana Sutherl...",female,48.0,1,0,39.6000,A16,C,11755,Duff Gordon,Lady
600,1.0,1,"Duff Gordon, Sir. Cosmo Edmund (""Mr Morgan"")",male,49.0,1,0,56.9292,A20,C,17485,Duff Gordon,Sir


In [50]:
all_data['Title'] = all_data['Title'].replace(['Lady'],'Mrs')
all_data['Title'] = all_data['Title'].replace(['Sir'],'Mr')

In [51]:
all_data[all_data.Title == 'the Countess']

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked,Ticket Number,Family Name,Title
PassengerId,,,,,,,,,,,,,
760,1.0,1,"Rothes, the Countess. of (Lucy Noel Martha Dye...",female,33.0,0,0,86.5,B77,S,110152,Rothes,the Countess


In [52]:
all_data[all_data['Family Name']== 'Rothes']

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked,Ticket Number,Family Name,Title
PassengerId,,,,,,,,,,,,,
760,1.0,1,"Rothes, the Countess. of (Lucy Noel Martha Dye...",female,33.0,0,0,86.5,B77,S,110152,Rothes,the Countess


In [53]:
all_data['Title'] = all_data['Title'].replace(['the Countess'],'Miss')

In [54]:
all_data[all_data.Title == 'Dr']

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked,Ticket Number,Family Name,Title
PassengerId,,,,,,,,,,,,,
246,0.0,1,"Minahan, Dr. William Edward",male,44.0,2,0,90.0000,C78,Q,19928,Minahan,Dr
318,0.0,2,"Moraweck, Dr. Ernest",male,54.0,0,0,14.0000,NaN,S,29011,Moraweck,Dr
399,0.0,2,"Pain, Dr. Alfred",male,23.0,0,0,10.5000,NaN,S,244278,Pain,Dr
633,1.0,1,"Stahelin-Maeglin, Dr. Max",male,32.0,0,0,30.5000,B50,C,13214,Stahelin-Maeglin,Dr
661,1.0,1,"Frauenthal, Dr. Henry William",male,50.0,2,0,133.6500,NaN,S,17611,Frauenthal,Dr
767,0.0,1,"Brewe, Dr. Arthur Jackson",male,NaN,0,0,39.6000,NaN,C,112379,Brewe,Dr
797,1.0,1,"Leader, Dr. Alice (Farnham)",female,49.0,0,0,25.9292,D17,S,17465,Leader,Dr
1185,NaN,1,"Dodge, Dr. Washington",male,53.0,1,1,81.8583,A34,S,33638,Dodge,Dr


In [55]:
all_data[all_data['Family Name']== 'Leader']

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked,Ticket Number,Family Name,Title
PassengerId,,,,,,,,,,,,,
797,1.0,1,"Leader, Dr. Alice (Farnham)",female,49.0,0,0,25.9292,D17,S,17465,Leader,Dr


In [56]:
all_data.loc[797,"Title"] = 'Miss'

In [57]:
all_data[all_data.Title == 'Dr']

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked,Ticket Number,Family Name,Title
PassengerId,,,,,,,,,,,,,
246,0.0,1,"Minahan, Dr. William Edward",male,44.0,2,0,90.0000,C78,Q,19928,Minahan,Dr
318,0.0,2,"Moraweck, Dr. Ernest",male,54.0,0,0,14.0000,NaN,S,29011,Moraweck,Dr
399,0.0,2,"Pain, Dr. Alfred",male,23.0,0,0,10.5000,NaN,S,244278,Pain,Dr
633,1.0,1,"Stahelin-Maeglin, Dr. Max",male,32.0,0,0,30.5000,B50,C,13214,Stahelin-Maeglin,Dr
661,1.0,1,"Frauenthal, Dr. Henry William",male,50.0,2,0,133.6500,NaN,S,17611,Frauenthal,Dr
767,0.0,1,"Brewe, Dr. Arthur Jackson",male,NaN,0,0,39.6000,NaN,C,112379,Brewe,Dr
1185,NaN,1,"Dodge, Dr. Washington",male,53.0,1,1,81.8583,A34,S,33638,Dodge,Dr


In [58]:
all_data['Title'] = all_data['Title'].replace(['Dr'],'Mr')

In [59]:
all_data.pivot_table('Age',index='Title',aggfunc=['min','mean','max','count'])

,min,mean,max,count
,Age,Age,Age,Age
Title,,,,
Master,0.33,5.482642,14.5,53
Miss,0.17,22.011991,63.0,216
Mr,11.00,32.670854,80.0,597
Mrs,14.00,37.069767,76.0,172
Rev,27.00,41.250000,57.0,8


In [60]:
all_data[all_data.Title=='Rev']

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked,Ticket Number,Family Name,Title
PassengerId,,,,,,,,,,,,,
150,0.0,2,"Byles, Rev. Thomas Roussel Davids",male,42.0,0,0,13.000,NaN,S,244310,Byles,Rev
151,0.0,2,"Bateman, Rev. Robert James",male,51.0,0,0,12.525,NaN,S,1166,Bateman,Rev
250,0.0,2,"Carter, Rev. Ernest Courtenay",male,54.0,1,0,26.000,NaN,S,244252,Carter,Rev
627,0.0,2,"Kirkland, Rev. Charles Leonard",male,57.0,0,0,12.350,NaN,Q,219533,Kirkland,Rev
849,0.0,2,"Harper, Rev. John",male,28.0,0,1,33.000,NaN,S,248727,Harper,Rev
887,0.0,2,"Montvila, Rev. Juozas",male,27.0,0,0,13.000,NaN,S,211536,Montvila,Rev
1041,NaN,2,"Lahtinen, Rev. William",male,30.0,1,1,26.000,NaN,S,250651,Lahtinen,Rev
1056,NaN,2,"Peruschitz, Rev. Joseph Maria",male,41.0,0,0,13.000,NaN,S,237393,Peruschitz,Rev


### I guess that two remaining revenants didn't survived too.

### Trying to see who traveled alone

In [61]:
all_data['Mix'] = all_data['Embarked'] +'-'+all_data['Pclass'].astype(str) + '-' +all_data['Ticket Number'].astype(str)+'-'+all_data['Family Name']

In [62]:
all_data.groupby(['Mix']).ngroups

1023

In [77]:
size_of_groups = all_data.groupby(['Mix'])['Title'].count().to_frame()

In [78]:
size_of_groups.head()

,Title
Mix,
C-1-110813-Warren,2
C-1-111361-Hippach,2
C-1-111369-Behr,1
C-1-111426-Homer,1
C-1-112277-Blank,1


In [79]:
size_of_groups['FamilySize'] = size_of_groups['Title']

In [80]:
size_of_groups.head()

,Title,FamilySize
Mix,,
C-1-110813-Warren,2,2
C-1-111361-Hippach,2,2
C-1-111369-Behr,1,1
C-1-111426-Homer,1,1
C-1-112277-Blank,1,1


In [81]:
size_of_groups.drop('Title',axis=1,inplace=True)

In [82]:
size_of_groups.head()

,FamilySize
Mix,
C-1-110813-Warren,2
C-1-111361-Hippach,2
C-1-111369-Behr,1
C-1-111426-Homer,1
C-1-112277-Blank,1


In [83]:
size_of_groups.reset_index()

,Mix,FamilySize
0,C-1-110813-Warren,2
1,C-1-111361-Hippach,2
2,C-1-111369-Behr,1
3,C-1-111426-Homer,1
4,C-1-112277-Blank,1
...,...,...
1018,S-3-7598-Dahl,1
1019,S-3-8471-Gronnestad,1
1020,S-3-8475-Kalvik,1
1021,S-3-851-van Billiard,3


In [84]:
all_data2 = all_data.merge(size_of_groups,left_on='Mix',right_on='Mix',how='left')

In [85]:
all_data2.head()

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked,Ticket Number,Family Name,Title,Mix,FamilySize
0,0.0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,7.2500,NaN,S,21171,Braund,Mr,S-3-21171-Braund,1.0
1,1.0,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,71.2833,C85,C,17599,Cumings,Mrs,C-1-17599-Cumings,2.0
2,1.0,3,"Heikkinen, Miss. Laina",female,26.0,0,0,7.9250,NaN,S,3101282,Heikkinen,Miss,S-3-3101282-Heikkinen,1.0
3,1.0,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,53.1000,C123,S,113803,Futrelle,Mrs,S-1-113803-Futrelle,2.0
4,0.0,3,"Allen, Mr. William Henry",male,35.0,0,0,8.0500,NaN,S,373450,Allen,Mr,S-3-373450-Allen,1.0


In [86]:
all_data2[all_data2['FamilySize']>1].head()

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked,Ticket Number,Family Name,Title,Mix,FamilySize
1,1.0,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,71.2833,C85,C,17599,Cumings,Mrs,C-1-17599-Cumings,2.0
3,1.0,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,53.1000,C123,S,113803,Futrelle,Mrs,S-1-113803-Futrelle,2.0
7,0.0,3,"Palsson, Master. Gosta Leonard",male,2.0,3,1,21.0750,NaN,S,349909,Palsson,Master,S-3-349909-Palsson,5.0
8,1.0,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,27.0,0,2,11.1333,NaN,S,347742,Johnson,Mrs,S-3-347742-Johnson,3.0
9,1.0,2,"Nasser, Mrs. Nicholas (Adele Achem)",female,14.0,1,0,30.0708,NaN,C,237736,Nasser,Mrs,C-2-237736-Nasser,2.0


In [87]:
all_data2.query("SibSp==0 and Parch==0 and FamilySize>1")

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked,Ticket Number,Family Name,Title,Mix,FamilySize
83,0.0,1,"Carrau, Mr. Francisco M",male,28.0,0,0,47.1000,NaN,S,113059,Carrau,Mr,S-1-113059-Carrau,2.0
161,1.0,2,"Watt, Mrs. James (Elizabeth ""Bessie"" Inglis Mi...",female,40.0,0,0,15.7500,NaN,S,33595,Watt,Mrs,S-2-33595-Watt,2.0
538,0.0,3,"Risien, Mr. Samuel Beard",male,NaN,0,0,14.5000,NaN,S,364498,Risien,Mr,S-3-364498-Risien,2.0
692,1.0,3,"Lam, Mr. Ali",male,NaN,0,0,56.4958,NaN,S,1601,Lam,Mr,S-3-1601-Lam,2.0
826,0.0,3,"Lam, Mr. Len",male,NaN,0,0,56.4958,NaN,S,1601,Lam,Mr,S-3-1601-Lam,2.0
1011,NaN,2,"Watt, Miss. Bertha J",female,12.0,0,0,15.7500,NaN,S,33595,Watt,Miss,S-2-33595-Watt,2.0
1253,NaN,2,"Ware, Mrs. John James (Florence Louise Long)",female,31.0,0,0,21.0000,NaN,S,31352,Ware,Mrs,S-2-31352-Ware,2.0
1273,NaN,3,"Risien, Mrs. Samuel (Emma)",female,NaN,0,0,14.5000,NaN,S,364498,Risien,Mrs,S-3-364498-Risien,2.0
1294,NaN,1,"Carrau, Mr. Jose Pedro",male,17.0,0,0,47.1000,NaN,S,113059,Carrau,Mr,S-1-113059-Carrau,2.0


### !!! We see that columns SibSp and Parch contains errors !!!

In [88]:
all_data2[all_data2['Family Name']=='Duff Gordon']

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked,Ticket Number,Family Name,Title,Mix,FamilySize
556,1.0,1,"Duff Gordon, Lady. (Lucille Christiana Sutherl...",female,48.0,1,0,39.6000,A16,C,11755,Duff Gordon,Mrs,C-1-11755-Duff Gordon,1.0
599,1.0,1,"Duff Gordon, Sir. Cosmo Edmund (""Mr Morgan"")",male,49.0,1,0,56.9292,A20,C,17485,Duff Gordon,Mr,C-1-17485-Duff Gordon,1.0


### Also, we see that Duff Gordon's are in the same family altho they have different ticket numbers

In [89]:
all_data2['Fsize'] = all_data2['SibSp'] + all_data2['Parch']+1

In [90]:
all_data2["Family Size"] = all_data2[["FamilySize", "Fsize"]].max(axis=1)

In [91]:
all_data2.head()

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked,Ticket Number,Family Name,Title,Mix,FamilySize,Fsize,Family Size
0,0.0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,7.2500,NaN,S,21171,Braund,Mr,S-3-21171-Braund,1.0,2,2.0
1,1.0,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,71.2833,C85,C,17599,Cumings,Mrs,C-1-17599-Cumings,2.0,2,2.0
2,1.0,3,"Heikkinen, Miss. Laina",female,26.0,0,0,7.9250,NaN,S,3101282,Heikkinen,Miss,S-3-3101282-Heikkinen,1.0,1,1.0
3,1.0,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,53.1000,C123,S,113803,Futrelle,Mrs,S-1-113803-Futrelle,2.0,2,2.0
4,0.0,3,"Allen, Mr. William Henry",male,35.0,0,0,8.0500,NaN,S,373450,Allen,Mr,S-3-373450-Allen,1.0,1,1.0


In [93]:
all_data2.drop(['Fsize','FamilySize'],axis=1,inplace=True)

In [94]:
all_data2.head()

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked,Ticket Number,Family Name,Title,Mix,Family Size
0,0.0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,7.2500,NaN,S,21171,Braund,Mr,S-3-21171-Braund,2.0
1,1.0,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,71.2833,C85,C,17599,Cumings,Mrs,C-1-17599-Cumings,2.0
2,1.0,3,"Heikkinen, Miss. Laina",female,26.0,0,0,7.9250,NaN,S,3101282,Heikkinen,Miss,S-3-3101282-Heikkinen,1.0
3,1.0,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,53.1000,C123,S,113803,Futrelle,Mrs,S-1-113803-Futrelle,2.0
4,0.0,3,"Allen, Mr. William Henry",male,35.0,0,0,8.0500,NaN,S,373450,Allen,Mr,S-3-373450-Allen,1.0


In [95]:
all_data2.rename(columns = {'Family Size': 'FamilySize','Family Name':'FamilyName', 'Ticket Number': 'TicketNumber'},inplace=True)

In [96]:
all_data2.query("FamilySize==1 and Title=='Master'")

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked,TicketNumber,FamilyName,Title,Mix,FamilySize
1230,NaN,3,"Betros, Master. Seman",male,NaN,0,0,7.2292,NaN,C,2622,Betros,Master,C-3-2622-Betros,1.0


In [132]:
all_data2[all_data2.FamilyName=='Betros']

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked,TicketNumber,FamilyName,Title,Mix,FamilySize
378,0.0,3,"Betros, Mr. Tannous",male,20.0,0,0,4.0125,NaN,C,2648,Betros,Mr,C-3-2648-Betros,2.0
1230,NaN,3,"Betros, Master. Seman",male,NaN,0,0,7.2292,NaN,C,2622,Betros,Master,C-3-2622-Betros,2.0


In [131]:
all_data2.loc[378,'FamilySize']=2
all_data2.loc[1230,'FamilySize']=2

In [99]:
all_data2.query("FamilySize==1 and Title=='Miss' and Age<15")

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked,TicketNumber,FamilyName,Title,Mix,FamilySize
14,0.0,3,"Vestrom, Miss. Hulda Amanda Adolfina",female,14.0,0,0,7.8542,NaN,S,350406,Vestrom,Miss,S-3-350406-Vestrom,1.0
777,1.0,3,"Emanuel, Miss. Virginia Ethel",female,5.0,0,0,12.4750,NaN,S,364516,Emanuel,Miss,S-3-364516-Emanuel,1.0
780,1.0,3,"Ayoub, Miss. Banoura",female,13.0,0,0,7.2292,NaN,C,2687,Ayoub,Miss,C-3-2687-Ayoub,1.0


### I asume that Ethel didn't traveled alone.

In [100]:
all_data2.query("FamilyName=='Emanuel'")

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked,TicketNumber,FamilyName,Title,Mix,FamilySize
777,1.0,3,"Emanuel, Miss. Virginia Ethel",female,5.0,0,0,12.475,NaN,S,364516,Emanuel,Miss,S-3-364516-Emanuel,1.0


In [101]:
all_data2.query("TicketNumber==364516")

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked,TicketNumber,FamilyName,Title,Mix,FamilySize
79,1.0,3,"Dowdell, Miss. Elizabeth",female,30.0,0,0,12.475,NaN,S,364516,Dowdell,Miss,S-3-364516-Dowdell,1.0
777,1.0,3,"Emanuel, Miss. Virginia Ethel",female,5.0,0,0,12.475,NaN,S,364516,Emanuel,Miss,S-3-364516-Emanuel,1.0


### We see that Ethel have same ticket number as Miss Elizabeth

In [102]:
all_data2.loc[79,'FamilySize']=2
all_data2.loc[777,'FamilySize']=2

In [103]:
all_data2.query("FamilyName=='Ayoub'")

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked,TicketNumber,FamilyName,Title,Mix,FamilySize
780,1.0,3,"Ayoub, Miss. Banoura",female,13.0,0,0,7.2292,NaN,C,2687,Ayoub,Miss,C-3-2687-Ayoub,1.0


In [104]:
all_data2.query("TicketNumber==2687")

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked,TicketNumber,FamilyName,Title,Mix,FamilySize
780,1.0,3,"Ayoub, Miss. Banoura",female,13.0,0,0,7.2292,NaN,C,2687,Ayoub,Miss,C-3-2687-Ayoub,1.0


In [105]:
all_data2.query("FamilyName=='Vestrom'")

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked,TicketNumber,FamilyName,Title,Mix,FamilySize
14,0.0,3,"Vestrom, Miss. Hulda Amanda Adolfina",female,14.0,0,0,7.8542,NaN,S,350406,Vestrom,Miss,S-3-350406-Vestrom,1.0


In [106]:
all_data2.query("TicketNumber==350406")

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked,TicketNumber,FamilyName,Title,Mix,FamilySize
14,0.0,3,"Vestrom, Miss. Hulda Amanda Adolfina",female,14.0,0,0,7.8542,NaN,S,350406,Vestrom,Miss,S-3-350406-Vestrom,1.0


# Let's input missing values for Age column

In [91]:
corr_matrix = titanic2.corr()

In [92]:
corr_matrix['Survived'].sort_values(ascending=False)

Survived        1.000000
Fare            0.257307
Parch           0.081629
FamilySize      0.020503
SibSp          -0.035322
Age            -0.077221
TicketNumber   -0.095605
Pclass         -0.338481
Name: Survived, dtype: float64

In [107]:
all_data2.pivot_table('Age',index=['FamilySize','Title'],aggfunc=['min','mean','median','max','count'])

min       mean median   max count
                     Age        Age    Age   Age   Age
FamilySize Title                                      
1.0        Master    NaN        NaN    NaN   NaN     0
           Miss    13.00  27.670833   26.0  58.0   120
           Mr      11.00  32.029002   29.0  80.0   431
           Mrs     18.00  41.000000   40.5  63.0    26
           Rev     27.00  43.600000   42.0  57.0     5
2.0        Master   0.42   5.650000    6.0  12.0     5
           Miss     2.00  19.850000   18.5  63.0    30
           Mr      17.00  34.635417   32.0  67.0    96
           Mrs     14.00  35.974026   35.0  76.0    77
           Rev     28.00  41.000000   41.0  54.0     2
3.0        Master   0.33   4.677727    3.5  13.0    22
           Miss     1.00  14.258065   10.0  39.0    31
           Mr      15.00  34.877551   34.0  70.0    49
           Mrs     16.00  36.911111   35.0  64.0    45
           Rev     30.00  30.000000   30.0  30.0     1
4.0        Master   0.92   3.480000    1.0  11.0     4
           Miss     0.17   7.772667    4.0  24.0    15
           Mr      23.00  32.000000   30.5  49.0     8
           Mrs     22.00  30.916667   31.5  48.0    12
5.0        Master   2.00   7.000000    6.0  13.0     3
           Miss     3.00  13.333333   13.5  21.0     6
           Mr      16.00  31.000000   23.5  61.0     4
           Mrs     29.00  44.750000   48.0  54.0     4
6.0        Master   1.00   5.500000    5.5  10.0    10
           Miss     2.00  17.200000   23.0  28.0     5
           Mr      14.00  30.600000   19.0  64.0     5
           Mrs     24.00  41.800000   41.0  60.0     5
7.0        Master   3.00   6.800000    5.0  13.0     5
           Miss     2.00  12.571429    9.0  38.0     7
           Mr      39.00  39.500000   39.5  40.0     2
           Mrs     38.00  38.500000   38.5  39.0     2
8.0        Master   1.00   7.000000    9.0  11.0     3
           Miss    10.00  13.000000   13.0  16.0     2
           Mr      14.00  27.000000   27.0  40.0     2
           Mrs     43.00  43.000000   43.0  43.0     1
11.0       Master  14.50  14.500000   14.5  14.5     1
           Miss      NaN        NaN    NaN   NaN     0
           Mr        NaN        NaN    NaN   NaN     0
           Mrs       NaN        NaN    NaN   NaN     0

Now, we are going to input missing data to all_data, acording, to above values.

In [108]:
table = all_data2.pivot_table('Age',index=['FamilySize','Title'],aggfunc=['mean'])

In [109]:
titles = ['Master','Miss','Mr','Mrs']

In [133]:
for i in range(2,8,1):
    for t in titles:
        all_data2.loc[(all_data2['FamilySize'] == i) & (all_data2['Title']==t) & (all_data2.Age.isnull()), 'Age'] = round(table.loc[(i, t),'mean'].Age)

In [111]:
for t in ['Miss','Mr','Mrs']:
    all_data2.loc[(all_data2['FamilySize'] == 1) & (all_data2['Title']==t) & (all_data2.Age.isnull()), 'Age'] = round(table.loc[(1, t),'mean'].Age)

In [128]:
all_data2[all_data2['Age'].isnull()]

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked,TicketNumber,FamilyName,Title,Mix,FamilySize
1230,NaN,3,"Betros, Master. Seman",male,NaN,0,0,7.2292,NaN,C,2622,Betros,Master,C-3-2622-Betros,1.0


In [113]:
all_data2[all_data2['FamilySize']==8]

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked,TicketNumber,FamilyName,Title,Mix,FamilySize
59,0.0,3,"Goodwin, Master. William Frederick",male,11.0,5,2,46.9,NaN,S,2144,Goodwin,Master,S-3-2144-Goodwin,8.0
71,0.0,3,"Goodwin, Miss. Lillian Amy",female,16.0,5,2,46.9,NaN,S,2144,Goodwin,Miss,S-3-2144-Goodwin,8.0
386,0.0,3,"Goodwin, Master. Sidney Leonard",male,1.0,5,2,46.9,NaN,S,2144,Goodwin,Master,S-3-2144-Goodwin,8.0
480,0.0,3,"Goodwin, Master. Harold Victor",male,9.0,5,2,46.9,NaN,S,2144,Goodwin,Master,S-3-2144-Goodwin,8.0
678,0.0,3,"Goodwin, Mrs. Frederick (Augusta Tyler)",female,43.0,1,6,46.9,NaN,S,2144,Goodwin,Mrs,S-3-2144-Goodwin,8.0
683,0.0,3,"Goodwin, Mr. Charles Edward",male,14.0,5,2,46.9,NaN,S,2144,Goodwin,Mr,S-3-2144-Goodwin,8.0
1030,NaN,3,"Goodwin, Mr. Charles Frederick",male,40.0,1,6,46.9,NaN,S,2144,Goodwin,Mr,S-3-2144-Goodwin,8.0
1031,NaN,3,"Goodwin, Miss. Jessie Allis",female,10.0,5,2,46.9,NaN,S,2144,Goodwin,Miss,S-3-2144-Goodwin,8.0


Here we input some values for Sage family

In [114]:
all_data2.loc[1233,'Age'] = 40

In [115]:
all_data2.loc[1256,'Age'] = 41

In [116]:
all_data2.loc[324,'Age'] = 10
all_data2.loc[201,'Age'] = 13
all_data2.loc[846,'Age'] = 22

In [117]:
all_data2.loc[863,'Age'] = 1
all_data2.loc[792,'Age'] = 11
all_data2.loc[180,'Age'] = 21
all_data2.loc[1079,'Age'] = 16

In [118]:
all_data2.loc[159,'Age'] = 2

In [147]:
all_data2.isnull().sum()

Survived             418
Pclass                 0
Name                   0
Sex                    0
Age                    0
SibSp                  0
Parch                  0
Fare                   0
Cabin               1014
Embarked               0
TicketNumber           0
FamilyName             0
Title                  0
Mix                    0
NumberPassengers       0
FamilySize             0
dtype: int64

### Now, we will input Mix and Fare values

In [135]:
all_data2[all_data2['Mix'].isnull()]

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked,TicketNumber,FamilyName,Title,Mix,FamilySize


### Bouth passengers have the same ticket number and same fare and they share cabin, we will asume that they traveld together

In [121]:
all_data2.pivot_table('Fare',index=['Pclass','Embarked'],aggfunc=['min','mean','max','count'])

min        mean       max count
                    Fare        Fare      Fare  Fare
Pclass Embarked                                     
1      C         25.7417  106.845330  512.3292   141
       Q         90.0000   90.000000   90.0000     3
       S          0.0000   72.148094  263.0000   177
2      C         12.0000   23.300593   41.5792    28
       Q          9.6875   11.735114   12.3500     7
       S          0.0000   21.206921   73.5000   242
3      C          4.0125   11.021624   22.3583   101
       Q          6.7500   10.390820   29.1250   113
       S          0.0000   14.435422   69.5500   494

In [122]:
all_data2.loc[(all_data2['Embarked']=='C') & (all_data2['Pclass']==1),'Fare'].describe()

count    141.000000
mean     106.845330
std       96.837527
min       25.741700
25%       49.504200
50%       76.729200
75%      134.500000
max      512.329200
Name: Fare, dtype: float64

In [123]:
all_data2.query('TicketNumber > 113500 and TicketNumber<120000 and Embarked =="C"')

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked,TicketNumber,FamilyName,Title,Mix,FamilySize
54,0.0,1,"Ostby, Mr. Engelhart Cornelius",male,65.0,0,1,61.9792,B30,C,113509,Ostby,Mr,C-1-113509-Ostby,2.0
377,0.0,1,"Widener, Mr. Harry Elkins",male,27.0,0,2,211.5000,C82,C,113503,Widener,Mr,C-1-113503-Widener,3.0
842,1.0,1,"Serepeca, Miss. Augusta",female,30.0,0,0,31.0000,NaN,C,113798,Serepeca,Miss,C-1-113798-Serepeca,1.0
917,NaN,1,"Ostby, Miss. Helene Ragnhild",female,22.0,0,1,61.9792,B36,C,113509,Ostby,Miss,C-1-113509-Ostby,2.0
965,NaN,1,"Geiger, Miss. Amalie",female,35.0,0,0,211.5000,C130,C,113503,Geiger,Miss,C-1-113503-Geiger,1.0
966,NaN,1,"Keeping, Mr. Edwin",male,32.5,0,0,211.5000,C132,C,113503,Keeping,Mr,C-1-113503-Keeping,1.0
1022,NaN,1,"Gracie, Col. Archibald IV",male,53.0,0,0,28.5000,C51,C,113780,Gracie,Mr,C-1-113780-Gracie,1.0
1109,NaN,1,"Widener, Mrs. George Dunton (Eleanor Elkins)",female,50.0,1,1,211.5000,C80,C,113503,Widener,Mrs,C-1-113503-Widener,3.0
1298,NaN,1,"Widener, Mr. George Dunton",male,50.0,1,1,211.5000,C80,C,113503,Widener,Mr,C-1-113503-Widener,3.0


In [124]:
all_data2.loc[61,'Embarked']='C'
all_data2.loc[829,'Embarked']='C'

In [125]:
all_data2.loc[61,'FamilySize']=2
all_data2.loc[829,'FamilySize']=2

In [126]:
all_data2.loc[61,'Mix']='C-1-113572-Icard'
all_data2.loc[829,'Mix']='C-1-113572-Stone'

In [136]:
all_data2.isnull().sum()

Survived         418
Pclass             0
Name               0
Sex                0
Age                0
SibSp              0
Parch              0
Fare               1
Cabin           1014
Embarked           0
TicketNumber       0
FamilyName         0
Title              0
Mix                0
FamilySize         0
dtype: int64

In [137]:
all_data2[all_data2['Fare'].isnull()]

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked,TicketNumber,FamilyName,Title,Mix,FamilySize
1043,NaN,3,"Storey, Mr. Thomas",male,60.5,0,0,NaN,NaN,S,3701,Storey,Mr,S-3-3701-Storey,1.0


In [138]:
all_data2.loc[(all_data2['Embarked']=='S') & (all_data2['Pclass']==3),'Fare'].describe()

count    494.000000
mean      14.435422
std       13.118281
min        0.000000
25%        7.854200
50%        8.050000
75%       15.900000
max       69.550000
Name: Fare, dtype: float64

In [139]:
all_data2.loc[1043,'Fare']=8

In [140]:
all_data2.isnull().sum()

Survived         418
Pclass             0
Name               0
Sex                0
Age                0
SibSp              0
Parch              0
Fare               0
Cabin           1014
Embarked           0
TicketNumber       0
FamilyName         0
Title              0
Mix                0
FamilySize         0
dtype: int64

In [141]:
all_data2.index = all_data2.index+1

In [142]:
all_data2.index

Int64Index([   1,    2,    3,    4,    5,    6,    7,    8,    9,   10,
            ...
            1300, 1301, 1302, 1303, 1304, 1305, 1306, 1307, 1308, 1309],
           dtype='int64', length=1309)

In [143]:
all_data2.rename(columns = {'FamilySize': 'NumberPassengers'},inplace=True)

In [144]:
all_data2['FamilySize']=all_data2['SibSp']+all_data2['Parch']+1

In [146]:
all_data2.head()

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked,TicketNumber,FamilyName,Title,Mix,NumberPassengers,FamilySize
1,0.0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,7.2500,NaN,S,21171,Braund,Mr,S-3-21171-Braund,2.0,2
2,1.0,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,71.2833,C85,C,17599,Cumings,Mrs,C-1-17599-Cumings,2.0,2
3,1.0,3,"Heikkinen, Miss. Laina",female,26.0,0,0,7.9250,NaN,S,3101282,Heikkinen,Miss,S-3-3101282-Heikkinen,1.0,1
4,1.0,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,53.1000,C123,S,113803,Futrelle,Mrs,S-1-113803-Futrelle,2.0,2
5,0.0,3,"Allen, Mr. William Henry",male,35.0,0,0,8.0500,NaN,S,373450,Allen,Mr,S-3-373450-Allen,1.0,1


In [148]:
train = all_data2[:891]

In [151]:
train.to_csv('train_prep.csv')

In [152]:
test = all_data2[891:]

In [153]:
test.to_csv('test_prep.csv')

In [158]:
train.pivot_table('Survived',index=['NumberPassengers','Sex'],aggfunc=['mean','sum'])

mean      sum
                         Survived Survived
NumberPassengers Sex                      
1.0              female  0.776860     94.0
                 male    0.155172     63.0
2.0              female  0.820225     73.0
                 male    0.240506     19.0
3.0              female  0.788462     41.0
                 male    0.396226     21.0
4.0              female  0.842105     16.0
                 male    0.500000      5.0
5.0              female  0.250000      3.0
                 male    0.000000      0.0
6.0              female  0.375000      3.0
                 male    0.000000      0.0
7.0              female  0.375000      3.0
                 male    0.250000      1.0
8.0              female  0.000000      0.0
                 male    0.000000      0.0
11.0             female  0.000000      0.0
                 male    0.000000      0.0

In [155]:
train.query('NumberPassengers==1').pivot_table('Survived',index=['Sex','Pclass'],aggfunc=['mean','sum'])

mean      sum
               Survived Survived
Sex    Pclass                   
female 1       0.968750     31.0
       2       0.903226     28.0
       3       0.603448     35.0
male   1       0.337838     25.0
       2       0.097222      7.0
       3       0.119231     31.0

In [156]:
train.query('NumberPassengers==1').pivot_table('Survived',index=['Sex','Pclass','Title'],aggfunc=['mean','sum'])

mean      sum
                     Survived Survived
Sex    Pclass Title                   
female 1      Miss   0.965517     28.0
              Mrs    1.000000      3.0
       2      Miss   0.904762     19.0
              Mrs    0.900000      9.0
       3      Miss   0.584906     31.0
              Mrs    0.800000      4.0
male   1      Mr     0.337838     25.0
       2      Mr     0.102941      7.0
              Rev    0.000000      0.0
       3      Mr     0.119231     31.0

In [157]:
train.query('NumberPassengers==2').pivot_table('Survived',index=['Sex','Pclass'],aggfunc=['mean','sum'])

mean      sum
               Survived Survived
Sex    Pclass                   
female 1       1.000000     38.0
       2       0.900000     18.0
       3       0.548387     17.0
male   1       0.375000     12.0
       2       0.066667      1.0
       3       0.187500      6.0

In [515]:
train.query('NumberPassengers==2').pivot_table('Survived',index=['Sex','Pclass','Title'],aggfunc=['mean','sum'])

mean      sum
                      Survived Survived
Sex    Pclass Title                    
female 1      Miss    1.000000      7.0
              Mrs     1.000000     31.0
       2      Miss    1.000000      4.0
              Mrs     0.875000     14.0
       3      Miss    0.562500      9.0
              Mrs     0.533333      8.0
male   1      Mr      0.375000     12.0
       2      Mr      0.076923      1.0
              Rev     0.000000      0.0
       3      Master  1.000000      3.0
              Mr      0.103448      3.0

In [516]:
train.query('NumberPassengers==3').pivot_table('Survived',index=['Sex','Pclass','Title'],aggfunc=['mean','sum'])

mean      sum
                      Survived Survived
Sex    Pclass Title                    
female 1      Miss    1.000000      9.0
              Mrs     1.000000      7.0
       2      Miss    1.000000      6.0
              Mrs     0.875000      7.0
       3      Miss    0.600000      6.0
              Mrs     0.500000      6.0
male   1      Master  1.000000      1.0
              Mr      0.400000      4.0
       2      Master  1.000000      8.0
              Mr      0.000000      0.0
       3      Master  1.000000      6.0
              Mr      0.105263      2.0

In [160]:
cabin = all_data2[all_data2['Cabin'].notnull()]

In [163]:
all_data2['CabinWord'] = cabin['Cabin'].str[0]

In [168]:
all_data2.pivot_table('Fare',index=['CabinWord','Pclass'],aggfunc=['min','median','mean','max','count'])

min    median        mean       max count
                     Fare      Fare        Fare      Fare  Fare
CabinWord Pclass                                               
A         1        0.0000  35.07710   41.244314   81.8583    22
B         1        0.0000  82.26670  122.383078  512.3292    65
C         1       25.7000  86.28960  107.926598  263.0000    94
D         1       25.9292  53.10000   58.919065  113.2750    40
          2       12.8750  13.39585   13.595833   15.0458     6
E         1       25.5875  55.44170   63.464706  134.5000    34
          2       10.5000  11.42500   11.587500   13.0000     4
          3        8.0500  12.47500   11.000000   12.4750     3
F         2       10.5000  26.00000   23.423077   39.0000    13
          3        7.2292   7.65000    9.395838   22.3583     8
G         3       10.4625  16.70000   14.205000   16.7000     5
T         1       35.5000  35.50000   35.500000   35.5000     1

In [169]:
train = all_data2[:891]

In [171]:
train.pivot_table('Survived',index=['CabinWord','Pclass'],aggfunc=['mean','sum','count'])

mean      sum    count
                  Survived Survived Survived
CabinWord Pclass                            
A         1       0.466667      7.0       15
B         1       0.744681     35.0       47
C         1       0.593220     35.0       59
D         1       0.758621     22.0       29
          2       0.750000      3.0        4
E         1       0.720000     18.0       25
          2       0.750000      3.0        4
          3       1.000000      3.0        3
F         2       0.875000      7.0        8
          3       0.200000      1.0        5
G         3       0.500000      2.0        4
T         1       0.000000      0.0        1